In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import SGD

Using Theano backend.
Can not use cuDNN on context None: Disabled by dnn.enabled flag
Mapped name None to device cuda0: GeForce GTX 960M (0000:01:00.0)


In [19]:
train_dir = './dataset/train'
validation_dir = './dataset/validation'

im_size = 224
INIT_LR = 1e-5

vgg_conv = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

In [3]:
vgg_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [12]:
for layer in vgg_conv.layers[:-4]: # last 4 layers will be trainable
    layer.trainable = False

for layer in vgg_conv.layers:
    print(layer, layer.trainable)

print('total layers:', vgg_conv.layers.__len__())

<keras.engine.topology.InputLayer object at 0x7f20fc7689e8> False
<keras.layers.convolutional.Conv2D object at 0x7f20fc768ba8> False
<keras.layers.convolutional.Conv2D object at 0x7f2117fe0518> False
<keras.layers.pooling.MaxPooling2D object at 0x7f20eff41ac8> False
<keras.layers.convolutional.Conv2D object at 0x7f20efec9358> False
<keras.layers.convolutional.Conv2D object at 0x7f20efec9e10> False
<keras.layers.pooling.MaxPooling2D object at 0x7f21042a5d30> False
<keras.layers.convolutional.Conv2D object at 0x7f20efde3c18> False
<keras.layers.convolutional.Conv2D object at 0x7f20efdd9cc0> False
<keras.layers.convolutional.Conv2D object at 0x7f20efdbf668> False
<keras.layers.pooling.MaxPooling2D object at 0x7f20efd4c518> False
<keras.layers.convolutional.Conv2D object at 0x7f20efcf66d8> False
<keras.layers.convolutional.Conv2D object at 0x7f20efcd0a20> False
<keras.layers.convolutional.Conv2D object at 0x7f20efce3c88> False
<keras.layers.pooling.MaxPooling2D object at 0x7f20efc17e48> Fa

In [13]:
model = Sequential()
model.add(vgg_conv)

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 2050      
Total params: 40,407,874
Trainable params: 32,772,610
Non-trainable params: 7,635,264
_________________________________________________________________


In [15]:
# Data augmentation

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 20
val_batchsize = 20

# Data Generator for Training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(im_size, im_size),
        batch_size=train_batchsize,
        class_mode='categorical'
)

# Data Generator for Validation data
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(im_size, im_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False
)

Found 4000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [20]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(lr=INIT_LR),
    metrics=['acc']
)

# Train the Model
# NOTE that we have multiplied the steps_per_epoch by 2. This is because we are using data augmentation.
history = model.fit_generator(
      train_generator,
      steps_per_epoch=2*train_generator.samples/train_generator.batch_size ,
      epochs=40,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1
)

# Save the Model
model.save('da_last4_layers.h5')

# Plot the accuracy and loss curves
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Epoch 1/40
326/400 [=======================>......] - ETA: 1:13 - loss: 0.8217 - acc: 0.5216

KeyboardInterrupt: 